In [1]:
# Mount the drive to persit the model
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install the modules
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
#!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install --no-deps xformers==0.0.27 "trl<0.9.0" peft accelerate bitsandbytes # Downgrade xformers to 0.0.27
!pip install transformers datasets
!pip install triton
#!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-gm3pogfh/unsloth_91e31ab36cb444e8985b6b8320268b6b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-gm3pogfh/unsloth_91e31ab36cb444e8985b6b8320268b6b
  Resolved https://github.com/unslothai/unsloth.git to commit fb77505f8429566f5d21d6ea5318c342e8a67991
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install torch==2.3.1+cu121 torchvision==0.16.1+cu121 torchaudio==2.0.2+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install xformers==0.0.27

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.3.1%2Bcu121-cp310-cp310-linux_x86_64.whl (781.0 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.16.1%2Bcu121-cp310-cp310-linux_x86_64.whl (6.8 MB)
ERROR: Could not find a version that satisfies the requirement torchaudio==2.0.2+cu121 (from versions: 2.1.0+cu121, 2.1.1+cu121, 2.1.2+cu121, 2.2.0+cu121, 2.2.1+cu121, 2.2.2+cu121, 2.3.0+cu121, 2.3.1+cu121, 2.4.0+cu121, 2.4.1+cu121)
ERROR: No matching distribution found for torchaudio==2.0.2+cu121


In [4]:
# Import the libraries
# note: we must use at least L4 GPU
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [5]:
# Import the models
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/trn0.2M_test.json"
OUTPUT_PATH_DATASET = "formatted_trn0.2M.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


In [6]:
import json

def format_dataset(data_path, output_path):
    """Formats the dataset for fine-tuning a Q&A model, skipping items with empty content.

    Args:
        data_path: Path to the original dataset.
        output_path: Path to save the formatted dataset.
    """
    data = []
    # Reading the original dataset line by line
    with open(data_path, 'r') as f:
        for line in f:
            # Attempt to load each line as a JSON object
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line: {line.strip()} - Error: {e}")

    formatted_data = []
    # Formatting each item for fine-tuning
    for item in data:
        content = item.get("content", "")
        # Skip items with empty content
        if not content.strip():
            continue

        # Create a question based on the product title
        question = f"What is the {item.get('title', '')}?"

        formatted_item = {
            "instruction": question,  # Question to the model
            "input_text": "",  # No additional input needed in this context
            "response": content  # Expected response based on the product details
        }
        formatted_data.append(formatted_item)

    # Save the formatted dataset as a JSON file
    with open(output_path, 'w') as f:
        json.dump(formatted_data, f, indent=4)




In [7]:
# Example usage:
# format_dataset('path/to/your/input.jsonl', 'path/to/your/output.json')
format_dataset(DATA_PATH, OUTPUT_PATH_DATASET)


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [8]:
from transformers import AutoTokenizer
from datasets import load_dataset


# Definindo o template do prompt com base na estrutura dos seus dados
product_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Limit the response to items that we have in the imported dataset. If you don't know the answer, just say that you don't know.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Inicialize o tokenizer (ajuste o nome do modelo conforme necessário)
tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-bnb-4bit")

# Define o token de fim de sequência (EOS)
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    """Formata os exemplos de entrada com base no template do prompt."""
    # Extrai os campos relevantes dos exemplos
    instructions = examples["instruction"]
    inputs = examples["input_text"]
    outputs = examples["response"]
    texts = []

    # Formata cada exemplo com o template definido
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Usa o template para criar o prompt completo
        text = product_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)

    # Retorna o texto formatado que será usado no treinamento
    return {"text": texts}


# Carrega o dataset usando o caminho especificado e seleciona o split de treinamento
dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split="train")

# Aplica a formatação dos prompts aos dados carregados
dataset = dataset.map(formatting_prompts_func, batched=True)

# Visualize alguns exemplos formatados (opcional)
print(dataset[0])


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/141883 [00:00<?, ? examples/s]

{'instruction': 'What is the Girls Ballet Tutu Neon Pink?', 'input_text': '', 'response': 'High quality 3 layer ballet tutu. 12 inches in length', 'text': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Limit the response to items that we have in the imported dataset. If you don't know the answer, just say that you don't know.\n\n### Instruction:\nWhat is the Girls Ballet Tutu Neon Pink?\n\n### Input:\n\n\n### Response:\nHigh quality 3 layer ballet tutu. 12 inches in length<|end_of_text|>"}


In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [10]:
# Train the model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/141883 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 141,883 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.628400
2,3.895900
3,3.763500
4,3.909500
5,3.735500
6,3.508000
7,2.379300
8,2.689000
9,2.685600
10,2.811800


In [12]:
# Configura o modelo para o modo de inferência
FastLanguageModel.for_inference(model)

# Formata o prompt usando uma pergunta relevante sobre o produto
inputs = tokenizer(
    [
        product_prompt.format(
            "What is the Girls Ballet Tutu Neon Pink?",  # Exemplo de instrução sobre o produto
            "",  # O input_text é vazio conforme o exemplo do dataset formatado
            ""   # Resposta será gerada pelo modelo
        )
    ],
    return_tensors="pt"
).to("cuda")  # Move os tensores para a GPU para acelerar a inferência

# Gera a resposta usando o modelo ajustado
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# Decodifica a saída do modelo para um texto legível
response = tokenizer.batch_decode(outputs)

# Mostra a resposta gerada pelo modelo
print(response)


["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Limit the response to items that we have in the imported dataset. If you don't know the answer, just say that you don't know.\n\n### Instruction:\nWhat is the Girls Ballet Tutu Neon Pink?\n\n### Input:\n\n\n### Response:\nThis is a beautiful tutu, and it is made well. I ordered a size 12, and it fits my 4 year old daughter perfectly. She has long legs, and it is a little bit short on her, but it is perfect for a little one. It is also very easy to wash and dry"]


In [13]:
# Configura o modelo para o modo de inferência
FastLanguageModel.for_inference(model)

# Formata o prompt usando uma pergunta relevante sobre o produto
inputs = tokenizer(
    [
        product_prompt.format(
            "Tell me more about Oscar Niemeyer: Curves of Irreverence",  # Exemplo de instrução sobre o produto
            "",  # O input_text é vazio conforme o exemplo do dataset formatado
            ""   # Resposta será gerada pelo modelo
        )
    ],
    return_tensors="pt"
).to("cuda")  # Move os tensores para a GPU para acelerar a inferência

# Gera a resposta usando o modelo ajustado
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# Decodifica a saída do modelo para um texto legível
response = tokenizer.batch_decode(outputs)

# Mostra a resposta gerada pelo modelo
print(response)


["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Limit the response to items that we have in the imported dataset. If you don't know the answer, just say that you don't know.\n\n### Instruction:\nTell me more about Oscar Niemeyer: Curves of Irreverence\n\n### Input:\n\n\n### Response:\nOscar Niemeyer: Curves of Irreverence is a beautifully illustrated book that celebrates the life and work of one of the world's most influential architects. It is the first book in the new series of monographs by Phaidon that examines the lives and work of the world's most important and influential"]


In [14]:
# Configura o modelo para o modo de inferência
FastLanguageModel.for_inference(model)

# Formata o prompt usando uma pergunta relevante sobre o produto
inputs = tokenizer(
    [
        product_prompt.format(
            "For who is indicated the Bach's Solo Violin Works: A Performer's Guide",  # Exemplo de instrução sobre o produto
            "",  # O input_text é vazio conforme o exemplo do dataset formatado
            ""   # Resposta será gerada pelo modelo
        )
    ],
    return_tensors="pt"
).to("cuda")  # Move os tensores para a GPU para acelerar a inferência

# Gera a resposta usando o modelo ajustado
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# Decodifica a saída do modelo para um texto legível
response = tokenizer.batch_decode(outputs)

# Mostra a resposta gerada pelo modelo
print(response)


["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Limit the response to items that we have in the imported dataset. If you don't know the answer, just say that you don't know.\n\n### Instruction:\nFor who is indicated the Bach's Solo Violin Works: A Performer's Guide\n\n### Input:\n\n\n### Response:\nThe book is divided into three parts. The first contains the technical and stylistic aspects of the works, with a discussion of the bowing and fingering issues of the pieces. The second part contains a detailed analysis of each piece, and the third part contains a brief biography of each composer and a discussion of the works"]


In [15]:
# Configura o modelo para o modo de inferência
FastLanguageModel.for_inference(model)

# Formata o prompt usando uma pergunta relevante sobre o produto
inputs = tokenizer(
    [
        product_prompt.format(
            "What you know about Facas Ginsu",  # Exemplo de instrução sobre o produto
            "",  # O input_text é vazio conforme o exemplo do dataset formatado
            ""   # Resposta será gerada pelo modelo
        )
    ],
    return_tensors="pt"
).to("cuda")  # Move os tensores para a GPU para acelerar a inferência

# Gera a resposta usando o modelo ajustado
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# Decodifica a saída do modelo para um texto legível
response = tokenizer.batch_decode(outputs)

# Mostra a resposta gerada pelo modelo
print(response)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Limit the response to items that we have in the imported dataset. If you don\'t know the answer, just say that you don\'t know.\n\n### Instruction:\nWhat you know about Facas Ginsu\n\n### Input:\n\n\n### Response:\nThe Ginsu knives are made in Japan and are distributed by Ginsu International, Inc. The Ginsu knives are made of a special steel called "Super Alloy 420."<|end_of_text|>']


In [16]:
# Configura o modelo para o modo de inferência
FastLanguageModel.for_inference(model)

# Formata o prompt usando uma pergunta relevante sobre o produto
inputs = tokenizer(
    [
        product_prompt.format(
            "What you know about Facas Ginsu. Answer in portuguese.",  # Exemplo de instrução sobre o produto
            "",  # O input_text é vazio conforme o exemplo do dataset formatado
            ""   # Resposta será gerada pelo modelo
        )
    ],
    return_tensors="pt"
).to("cuda")  # Move os tensores para a GPU para acelerar a inferência

# Gera a resposta usando o modelo ajustado
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# Decodifica a saída do modelo para um texto legível
response = tokenizer.batch_decode(outputs)

# Mostra a resposta gerada pelo modelo
print(response)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Limit the response to items that we have in the imported dataset. If you don't know the answer, just say that you don't know.\n\n### Instruction:\nWhat you know about Facas Ginsu. Answer in portuguese.\n\n### Input:\n\n\n### Response:\nA faca Ginsu (ou faca Ginsu de 12,5 polegadas) foi uma faca de cozinha que foi vendida por vários anunciantes de televisão na década de 1970, como parte de uma série de produtos de artes marciais.<|end_of_text|>"]


In [12]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model")

('/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 02 - Fine tuning de LLM para documentos/lora_model/tokenizer.json')

In [ ]:
# generate a file for tests
#DATA_PATH = "trn0.1M_test.json"
#OUTPUT_PATH_DATASET = "formatted_trn0.1M_test.json"
#format_dataset(DATA_PATH, OUTPUT_PATH_DATASET)

In [ ]:
# Import the test file
#import json
#products_test = []

#with open('/content/formatted_trn0.1M_test.json', 'r') as products_file:
  #products_test = json.loads(products_file.read())


In [ ]:

# display the first line
#products_test[0]